
# Nettoyage des excels avant exportation vers Jasper

Charpeil Romain,
Wattelier Tonny

## Le fichier 'ValeursFoncieres2020'

In [ ]:
import pandas as pd

#Ce script est pour le nettoyage de ValeursFoncieres2020 avec export csv
cols_utiles = ["Surface terrain", "Valeur fonciere", "Code postal", "Commune", "Code departement"]
df_valeurs_foncieres = pd.read_excel("ValeursFoncieres2020.xlsx", usecols=cols_utiles)
print(df_valeurs_foncieres.head())

#Nettoyer le nom des colonnes
df_valeurs_foncieres.columns = df_valeurs_foncieres.columns.str.strip()
# Supprimer les lignes où une des colonnes importantes est manquante
df_valeurs_foncieres.dropna(subset=["Valeur fonciere", "Surface terrain", "Code departement"], inplace=True)
# Remplacer les NaN dans 'Valeur fonciere' par 0, et dans 'Surface terrain' par la moyenne
df_valeurs_foncieres["Valeur fonciere"].fillna(0, inplace=True)
df_valeurs_foncieres["Surface terrain"].fillna(df_valeurs_foncieres["Surface terrain"].mean(), inplace=True)
# Nettoyer les espaces dans les colonnes
df_valeurs_foncieres.columns = df_valeurs_foncieres.columns.str.strip()

# Nettoyer les espaces dans les colonnes textuelles (Commune, Code postal, etc.)
df_valeurs_foncieres["Commune"] = df_valeurs_foncieres["Commune"].str.strip()

# Convertir 'Code postal' en chaîne de caractères
df_valeurs_foncieres["Code postal"] = df_valeurs_foncieres["Code postal"].astype(str)

# Nettoyer les espaces
df_valeurs_foncieres["Code postal"] = df_valeurs_foncieres["Code postal"].str.strip()

# Remplacer les valeurs non convertibles ou manquantes par une chaîne vide
df_valeurs_foncieres["Code postal"] = df_valeurs_foncieres["Code postal"].fillna('').astype(str)
# Vérification des types de données et des premières lignes
print(df_valeurs_foncieres.info())
print(df_valeurs_foncieres.head())
df_valeurs_foncieres["Commune"] = df_valeurs_foncieres["Commune"].str.lower()
# Convertir les colonnes en numérique, avec gestion des erreurs (remplacer les erreurs par NaN)
df_valeurs_foncieres["Valeur fonciere"] = pd.to_numeric(df_valeurs_foncieres["Valeur fonciere"], errors='coerce')
df_valeurs_foncieres["Surface terrain"] = pd.to_numeric(df_valeurs_foncieres["Surface terrain"], errors='coerce')
#  Créer une liste des départements de l'Occitanie
departements_occitanie = [31, 32, 34, 46, 48, 65, 66, 81, 82]
# Filtrer les données pour la région Occitanie (départements 31 à 66)
df_valeurs_foncieres = df_valeurs_foncieres[df_valeurs_foncieres['Code departement'].isin(departements_occitanie)]
# Supprimer les doublons exacts
df_valeurs_foncieres.drop_duplicates(inplace=True)
# Afficher un résumé des données pour voir les types et vérifier les valeurs uniques
print(df_valeurs_foncieres.info())

# Vérifier quelques premières lignes après nettoyage
print(df_valeurs_foncieres.head())

# Définir le rendement locatif moyen (en pourcentage, ici 5% par an)
rendement_locatif_moyen = 0.05  # 5%

# Calculer le loyer mensuel estimé (en utilisant la valeur foncière)
df_valeurs_foncieres['Loyer mensuel estimé'] = (df_valeurs_foncieres['Valeur fonciere'] * rendement_locatif_moyen) / 12

# Calculer le rendement locatif avec loyer mensuel estimé et prix d'achat
df_valeurs_foncieres['Rendement locatif estimé'] = (df_valeurs_foncieres['Loyer mensuel estimé'] * 12) / df_valeurs_foncieres['Valeur fonciere'] * 100

# Afficher les résultats avec les rendements calculés
print(df_valeurs_foncieres[['Commune', 'Valeur fonciere', 'Loyer mensuel estimé', 'Rendement locatif estimé']].head())

# Exporter le DataFrame nettoyé et mis à jour au format CSV
df_valeurs_foncieres.to_csv("ValeursFoncieres_nettoyees.csv", index=False)

# Confirmation de l'exportation
print("Exportation terminée ! Le fichier CSV a été sauvegardé sous 'ValeursFoncieres_nettoyees.csv'.")

## Le fichier 'Revenu médian par ménage'

In [28]:
import pandas as pd

def nettoyer_feuille(df, nom_feuille):
    df = df.copy()  # Créer une copie explicite du DataFrame
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df.dropna(how='all')
    df.loc[:, 'source'] = nom_feuille  # Utiliser .loc pour l'assignation
    return df

fichier_excel = "Revenu médian par ménage.xlsx"
feuilles = pd.ExcelFile(fichier_excel).sheet_names

dataframes_nettoyes = []

for feuille in feuilles:
    df = pd.read_excel(fichier_excel, sheet_name=feuille)
    df_nettoye = nettoyer_feuille(df, feuille)
    dataframes_nettoyes.append(df_nettoye)

df_combine = pd.concat(dataframes_nettoyes, ignore_index=True)

nom_fichier_csv = "Revenu_median_par_menage_nettoye.csv"
df_combine.to_csv(nom_fichier_csv, index=False, encoding='utf-8-sig')
print(f"Exportation terminée : {nom_fichier_csv}")


Exportation terminée : Revenu_median_par_menage_nettoye.csv


##Pour le fichier 'famille_POPULATION-STRUCTURE_14012025'

In [29]:
import pandas as pd
import numpy as np


def nettoyer_feuille(df, nom_feuille):
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df.dropna(how='all')
    df.loc[:, 'source'] = nom_feuille
    return df

fichier_excel = "famille_POPULATION-STRUCTURE_14012025.xlsx"
feuilles = pd.ExcelFile(fichier_excel).sheet_names

dataframes_nettoyes = []

for feuille in feuilles:
    df = pd.read_excel(fichier_excel, sheet_name=feuille)
    df_nettoye = nettoyer_feuille(df, feuille)
    dataframes_nettoyes.append(df_nettoye)

df_combine = pd.concat(dataframes_nettoyes, ignore_index=True)

# Nettoyage supplémentaire
df_combine = df_combine.replace('(O)', np.nan)
df_combine = df_combine.apply(lambda x: pd.to_numeric(x, errors='ignore'))

# Supprimer les lignes entièrement vides
df_combine = df_combine.dropna(how='all')

# Remplir les valeurs manquantes avec la moyenne de la colonne pour les colonnes numériques
for col in df_combine.select_dtypes(include=['float64', 'int64']).columns:
    df_combine[col].fillna(df_combine[col].mean(), inplace=True)

nom_fichier_csv = "famille_POPULATION-STRUCTURE_nettoye.csv"
df_combine.to_csv(nom_fichier_csv, index=False, encoding='utf-8-sig')
print(f"Exportation terminée : {nom_fichier_csv}")

# Afficher un aperçu des données nettoyées
print(df_combine.head())
print(df_combine.info())


/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
<ipython-input-29-f4731bc3347a>:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed i

Exportation terminée : famille_POPULATION-STRUCTURE_nettoye.csv
                                             libellé     idbank  \
0  Démographie - Population au début du mois - Fr...   436387.0   
1  Démographie - Population au début du mois - Fr...  1641607.0   
2  Population totale au 1er janvier - France métr...    67670.0   
3  Population totale au 1er janvier - France (inc...  1641586.0   
4  Population au 1er janvier - Hommes - Moins de ...  1686784.0   

  dernière_mise_à_jour  période  1975-01  1975-02  1975-03  1975-04  1975-05  \
0     14/01/2025 12:00      NaN  52600.0  52608.0  52623.0  52640.0  52662.0   
1     14/01/2025 12:00      NaN  52600.0  52608.0  52623.0  52640.0  52662.0   
2     14/01/2025 12:00      NaN  52600.0  52608.0  52623.0  52640.0  52662.0   
3     14/01/2025 12:00      NaN  52600.0  52608.0  52623.0  52640.0  52662.0   
4     14/01/2025 12:00      NaN  52600.0  52608.0  52623.0  52640.0  52662.0   

   1975-06  ...  zone_géographique  puissance  indic

## Pour le fichier 'age-insee-2020'

In [30]:
import pandas as pd
import numpy as np
import warnings

def nettoyer_feuille(df, nom_feuille):
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df.dropna(how='all')
    df.loc[:, 'source'] = nom_feuille
    return df

fichier_excel = "age-insee-2020.xlsx"

with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    feuilles = pd.ExcelFile(fichier_excel).sheet_names

dataframes_nettoyes = []

for feuille in feuilles:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        df = pd.read_excel(fichier_excel, sheet_name=feuille)
    df_nettoye = nettoyer_feuille(df, feuille)
    dataframes_nettoyes.append(df_nettoye)

df_combine = pd.concat(dataframes_nettoyes, ignore_index=True)

def safe_numeric_conversion(x):
    try:
        return pd.to_numeric(x)
    except ValueError:
        return x

df_combine = df_combine.replace('(O)', np.nan).infer_objects()
df_combine = df_combine.apply(safe_numeric_conversion)

df_combine = df_combine.dropna(how='all')

for col in df_combine.select_dtypes(include=['float64', 'int64']).columns:
    df_combine[col] = df_combine[col].fillna(df_combine[col].mean())

nom_fichier_csv = "age-insee-2020_nettoye.csv"
df_combine.to_csv(nom_fichier_csv, index=False, encoding='utf-8-sig')
print(f"Exportation terminée : {nom_fichier_csv}")

print(df_combine.head())
print(df_combine.info())


Exportation terminée : age-insee-2020_nettoye.csv
  insee                      nom       epci dep  reg        f0-2        f3-5  \
0  1001  L'Abergement-Clémenciat  200069193  D1  R84   13.414280   12.509227   
1  1002    L'Abergement-de-Varey  240100883  D1  R84    2.994218    6.050262   
2  1004        Ambérieu-en-Bugey  240100883  D1  R84  294.667755  245.153009   
3  1005      Ambérieux-en-Dombes  200042497  D1  R84   28.000000   33.000000   
4  1006                  Ambléon  200040350  D1  R84    0.991228    1.982456   

        f6-10      f11-17      f18-24  ...       h40-54      h55-64  \
0   19.214486   37.181506   14.062216  ...    81.849108   61.039016   
1   12.232163   11.868718    5.201595  ...    37.479209   10.977019   
2  382.800636  599.105269  680.830755  ...  1238.119870  782.771068   
3   60.000000   79.000000   50.000000  ...   183.000000  124.000000   
4    1.982456    0.991228    1.982456  ...    10.903509   13.877193   

       h65-79        h80+  source  unnamed

## Pour le fichier 'age-insee-2020(2)'

In [31]:
import pandas as pd
import numpy as np
import warnings

def nettoyer_feuille(df, nom_feuille):
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df.dropna(how='all')
    df.loc[:, 'source'] = nom_feuille
    return df

fichier_excel = "age-insee-2020(2).xlsx"

with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    feuilles = pd.ExcelFile(fichier_excel).sheet_names

dataframes_nettoyes = []

for feuille in feuilles:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        df = pd.read_excel(fichier_excel, sheet_name=feuille)
    df_nettoye = nettoyer_feuille(df, feuille)
    dataframes_nettoyes.append(df_nettoye)

df_combine = pd.concat(dataframes_nettoyes, ignore_index=True)

def safe_numeric_conversion(x):
    try:
        return pd.to_numeric(x)
    except ValueError:
        return x

df_combine = df_combine.replace('(O)', np.nan).infer_objects()
df_combine = df_combine.apply(safe_numeric_conversion)

df_combine = df_combine.dropna(how='all')

for col in df_combine.select_dtypes(include=['float64', 'int64']).columns:
    df_combine[col] = df_combine[col].fillna(df_combine[col].mean())

nom_fichier_csv = "age-insee-2020(2)_nettoye.csv"
df_combine.to_csv(nom_fichier_csv, index=False, encoding='utf-8-sig')
print(f"Exportation terminée : {nom_fichier_csv}")

print(df_combine.head())
print(df_combine.info())


Exportation terminée : age-insee-2020(2)_nettoye.csv
  insee                      nom       epci dep  reg        f0-2        f3-5  \
0  1001  L'Abergement-Clémenciat  200069193  D1  R84   13.414280   12.509227   
1  1002    L'Abergement-de-Varey  240100883  D1  R84    2.994218    6.050262   
2  1004        Ambérieu-en-Bugey  240100883  D1  R84  294.667755  245.153009   
3  1005      Ambérieux-en-Dombes  200042497  D1  R84   28.000000   33.000000   
4  1006                  Ambléon  200040350  D1  R84    0.991228    1.982456   

        f6-10      f11-17      f18-24  ...       h40-54      h55-64  \
0   19.214486   37.181506   14.062216  ...    81.849108   61.039016   
1   12.232163   11.868718    5.201595  ...    37.479209   10.977019   
2  382.800636  599.105269  680.830755  ...  1238.119870  782.771068   
3   60.000000   79.000000   50.000000  ...   183.000000  124.000000   
4    1.982456    0.991228    1.982456  ...    10.903509   13.877193   

       h65-79        h80+  source  unna

## Pour le fichier 'ValeursChomage'

In [32]:
import pandas as pd
import numpy as np
import warnings

def nettoyer_feuille(df, nom_feuille):
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df.dropna(how='all')
    df.loc[:, 'source'] = nom_feuille
    return df

fichier_excel = "ValeursChomage.xlsx"

with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    feuilles = pd.ExcelFile(fichier_excel).sheet_names

dataframes_nettoyes = []

for feuille in feuilles:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        df = pd.read_excel(fichier_excel, sheet_name=feuille)
    df_nettoye = nettoyer_feuille(df, feuille)
    dataframes_nettoyes.append(df_nettoye)

df_combine = pd.concat(dataframes_nettoyes, ignore_index=True)

def safe_numeric_conversion(x):
    try:
        return pd.to_numeric(x)
    except ValueError:
        return x

df_combine = df_combine.replace('(O)', np.nan).infer_objects()
df_combine = df_combine.apply(safe_numeric_conversion)

df_combine = df_combine.dropna(how='all')

for col in df_combine.select_dtypes(include=['float64', 'int64']).columns:
    df_combine[col] = df_combine[col].fillna(df_combine[col].mean())

nom_fichier_csv = "ValeursChomage_nettoye.csv"
df_combine.to_csv(nom_fichier_csv, index=False, encoding='utf-8-sig')
print(f"Exportation terminée : {nom_fichier_csv}")

print(df_combine.head())
print(df_combine.info())


Exportation terminée : ValeursChomage_nettoye.csv
  taux_de_chômage_localisé_par_région_et_département               source  \
0                                       Définition :  Sources_et_méthodes   
1  Le taux de chômage localisé est le rapport ent...  Sources_et_méthodes   
2                               Sources et mesures :  Sources_et_méthodes   
3  Les taux de chômage localisé sont issus d’une ...  Sources_et_méthodes   
4  Les principales étapes de calcul du taux de ch...  Sources_et_méthodes   

  estimations_de_taux_de_chômage_localisé_:_comparaisons_départementales  \
0                                                NaN                       
1                                                NaN                       
2                                                NaN                       
3                                                NaN                       
4                                                NaN                       

  unnamed:_1 unnamed:_2 unname

## Pour le fichier 'Taux-de-menage-locataire-proprietaire'

In [33]:
import pandas as pd
import numpy as np
import warnings

def nettoyer_feuille(df, nom_feuille):
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df.dropna(how='all')
    df.loc[:, 'source'] = nom_feuille
    return df

fichier_excel = "Taux-de-menage-locataire-proprietaire.xlsx"

with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    feuilles = pd.ExcelFile(fichier_excel).sheet_names

dataframes_nettoyes = []

for feuille in feuilles:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        df = pd.read_excel(fichier_excel, sheet_name=feuille)
    df_nettoye = nettoyer_feuille(df, feuille)
    dataframes_nettoyes.append(df_nettoye)

df_combine = pd.concat(dataframes_nettoyes, ignore_index=True)

def safe_numeric_conversion(x):
    try:
        return pd.to_numeric(x, errors='ignore')
    except (ValueError, TypeError):
        return x

df_combine = df_combine.replace('', np.nan)
df_combine = df_combine.apply(safe_numeric_conversion)

df_combine = df_combine.dropna(how='all')

for col in df_combine.select_dtypes(include=['float64', 'int64']).columns:
    df_combine[col] = df_combine[col].fillna(df_combine[col].mean())

nom_fichier_csv = "Taux-de-menage-locataire-proprietaire_nettoye.csv"
df_combine.to_csv(nom_fichier_csv, index=False, encoding='utf-8-sig')
print(f"Exportation terminée : {nom_fichier_csv}")

print(df_combine.head())
print(df_combine.info())

Exportation terminée : Taux-de-menage-locataire-proprietaire_nettoye.csv
  statut_d’occupation_des_résidences_principales unnamed:_1 unnamed:_2  \
0                                            NaN        NaN        NaN   
1                                 Au 1er janvier     2001.0     2004.0   
2                                  Propriétaires       55.9       56.6   
3                                  Non accédants       34.9       36.4   
4                                      Accédants       21.0       20.2   

  unnamed:_3 unnamed:_4 unnamed:_5 unnamed:_6 unnamed:_7    source  \
0        NaN        NaN        NaN        NaN       en %  Figure 1   
1     2007.0     2010.0     2013.0     2016.0       2018  Figure 1   
2       57.2       57.5       57.7       57.7       57.6  Figure 1   
3       37.7       37.8       37.9       37.8       37.7  Figure 1   
4       19.6       19.7       19.8       19.9       19.9  Figure 1   

  résidences_principales_selon_la_taille_des_ménages_et_le_st

<ipython-input-33-3f291b37866b>:31: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  return pd.to_numeric(x, errors='ignore')


## Fusion des fichiers nettoyés

In [37]:
import pandas as pd
import os

# Liste mise à jour des fichiers CSV nettoyés
fichiers = [
    "Revenu_median_par_menage_nettoye.csv",
    "age-insee-2020_nettoye.csv",
    "age-insee-2020(2)_nettoye.csv", # This is an Excel file
    "ValeursChomage_nettoye.csv",
    "Taux-de-menage-locataire-proprietaire_nettoye.csv",
    "ValeursFoncieres_nettoyees.csv",
    "famille_POPULATION-STRUCTURE_nettoye.csv"
]

# Initialisation d'un DataFrame vide
df_fusionne = pd.DataFrame()

# Boucle sur chaque fichier pour l'ajouter au DataFrame fusionné
for fichier in fichiers:
    if os.path.exists(fichier):
        if fichier.endswith('.csv'):
            df = pd.read_csv(fichier)
        elif fichier.endswith('.xlsx'):
            # Use pd.read_excel() for Excel files
            df = pd.read_excel(fichier)
        else:
            print(f"Ignoring file {fichier} with unsupported format.")
            continue  # Skip to the next file

        df_fusionne = pd.concat([df_fusionne, df], ignore_index=True)

# Sauvegarde du DataFrame fusionné dans un seul fichier CSV
fichier_sortie = "donnees_fusionnees_completes.csv"
df_fusionne.to_csv(fichier_sortie, index=False, encoding='utf-8-sig')

print(f"Fusion terminée. Le fichier '{fichier_sortie}' a été créé avec tous les fichiers, y compris ValeursFoncieres2020 et famille_POPULATION-STRUCTURE_14012025.")

<ipython-input-37-430568c9c8d9>:22: DtypeWarning: Columns (0,2,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fichier)
<ipython-input-37-430568c9c8d9>:22: DtypeWarning: Columns (0,2,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fichier)


Fusion terminée. Le fichier 'donnees_fusionnees_completes.csv' a été créé avec tous les fichiers, y compris ValeursFoncieres2020 et famille_POPULATION-STRUCTURE_14012025.
